In [3]:
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions.countDistinct
import org.apache.spark.sql.functions.collect_list
import org.apache.spark.sql.functions.count
import org.apache.spark.sql.functions._
import org.apache.spark.sql
import org.apache.spark.sql.types._
import org.apache.spark.sql.Column
import org.apache.spark.sql.Row


case class item(items: Array[Int])
case class candidate(value: Array[Int])

val dataRDD = sc.textFile("T10I4D100K.dat").map(row => row.split(" ").map(v => v.toInt).toArray).map(d => item(d))
val dataCount = dataRDD.count()
val support = 0.01 * dataCount

//rawDataRDD.collect.foreach(println)
//println(dataCount)
//println(support)



defined class item
defined class candidate
dataRDD = MapPartitionsRDD[3] at map at <console>:29
dataCount = 100000
support = 1000.0


1000.0

In [4]:
//val itemsRDD = dataRDD.zipWithIndex().map(d => item(d._2, d._1))
//println(itemsRDD.take(1)(0))

Name: Syntax Error.
Message: 
StackTrace: 

In [5]:
val candidates = dataRDD.map(row => row.items.map(item => (item) ))
    .flatMap(y => y)
    .distinct
    .map(c => candidate(Array(c)))
println(candidates.count())

870


candidates = MapPartitionsRDD[9] at map at <console>:47


MapPartitionsRDD[9] at map at <console>:47

In [6]:
val candidatesDF = candidates.toDF()
val dataDF = dataRDD.toDF()
val crossDF = dataDF.crossJoin(candidatesDF)
candidatesDF.show()
crossDF.show()

val singletonDF = crossDF.withColumn(
    "singleton",
    array_intersect( col("value"), col("items"))
)
singletonDF.show()
singletonDF.createOrReplaceTempView("singletons")
val sqlContext = new SQLContext(sc)
val frequent_singletons = sqlContext.sql("SELECT d.items, d.singleton FROM singletons d WHERE size(d.singleton) > 0 ")
frequent_singletons.show()

val freq_singletons = frequent_singletons.groupBy("singleton").agg(count("items")as "support").where(col("support") > support)
freq_singletons.show

//println(dup_filtered.count())


+-----+
|value|
+-----+
| [68]|
|[454]|
|[324]|
|[180]|
|[320]|
|[752]|
|[408]|
|[428]|
|[986]|
|[464]|
|[346]|
| [14]|
|[466]|
| [24]|
|[520]|
|[912]|
|[146]|
|[140]|
|[204]|
|[514]|
+-----+
only showing top 20 rows

+--------------------+-----+
|               items|value|
+--------------------+-----+
|[25, 52, 164, 240...| [68]|
|[25, 52, 164, 240...|[454]|
|[25, 52, 164, 240...|[324]|
|[25, 52, 164, 240...|[180]|
|[25, 52, 164, 240...|[320]|
|[25, 52, 164, 240...|[752]|
|[25, 52, 164, 240...|[408]|
|[25, 52, 164, 240...|[428]|
|[25, 52, 164, 240...|[986]|
|[25, 52, 164, 240...|[464]|
|[25, 52, 164, 240...|[346]|
|[25, 52, 164, 240...| [14]|
|[25, 52, 164, 240...|[466]|
|[25, 52, 164, 240...| [24]|
|[25, 52, 164, 240...|[520]|
|[25, 52, 164, 240...|[912]|
|[25, 52, 164, 240...|[146]|
|[25, 52, 164, 240...|[140]|
|[25, 52, 164, 240...|[204]|
|[25, 52, 164, 240...|[514]|
+--------------------+-----+
only showing top 20 rows

+--------------------+-----+---------+
|               items

candidatesDF = [value: array<int>]
dataDF = [items: array<int>]
crossDF = [items: array<int>, value: array<int>]
singletonDF = [items: array<int>, value: array<int> ... 1 more field]
sqlContext = org.apache.spark.sql.SQLContext@5960b6fa
frequent_singletons = [items: array<int>, singleton: array<int>]
freq_singletons = [singleton: array<int>, support: bigint]


[singleton: array<int>, support: bigint]

In [7]:
val candidate_2a = freq_singletons.select("singleton")
val candidate_2 = candidate_2a.crossJoin(candidate_2a.select(col("singleton") as "singleton2")).where($"singleton" < $"singleton2")

candidate_2.show

val crossDF2 = candidate_2.withColumn(
    "candidate2",
    concat(col("singleton"), col("singleton2"))
).crossJoin(dataDF).drop("singleton").drop("singleton2")
crossDF2.show


+---------+----------+
|singleton|singleton2|
+---------+----------+
|    [471]|     [496]|
|    [496]|     [540]|
|    [496]|     [623]|
|    [496]|     [897]|
|    [471]|     [540]|
|    [471]|     [623]|
|    [471]|     [897]|
|    [496]|     [516]|
|    [471]|     [516]|
|    [496]|     [580]|
|    [471]|     [580]|
|    [496]|     [883]|
|    [471]|     [883]|
|    [496]|     [970]|
|    [496]|     [804]|
|    [471]|     [970]|
|    [471]|     [804]|
|    [496]|     [853]|
|    [471]|     [472]|
|    [471]|     [853]|
+---------+----------+
only showing top 20 rows

+----------+--------------------+
|candidate2|               items|
+----------+--------------------+
|[471, 496]|[25, 52, 164, 240...|
|[471, 496]|[39, 120, 124, 20...|
|[471, 496]|[35, 249, 674, 71...|
|[471, 496]|[39, 422, 449, 70...|
|[471, 496]|[15, 229, 262, 28...|
|[471, 496]|[26, 104, 143, 32...|
|[471, 496]|[7, 185, 214, 350...|
|[471, 496]|          [227, 390]|
|[471, 496]|[71, 192, 208, 27...|
|[471, 496]|[1

candidate_2a = [singleton: array<int>]
candidate_2 = [singleton: array<int>, singleton2: array<int>]
crossDF2 = [candidate2: array<int>, items: array<int>]


[candidate2: array<int>, items: array<int>]

In [15]:
import org.apache.spark.sql.functions.lit

def contains_all(x: Column, y: Column): Column = {
    return size(array_intersect(x, y))
}
val dupletonDF = crossDF2.withColumn(
    "dupleton",
    contains_all( col("candidate2"), col("items"))
)
dupletonDF.show()
dupletonDF.createOrReplaceTempView("dupletons")



+----------+--------------------+--------+
|candidate2|               items|dupleton|
+----------+--------------------+--------+
|[471, 496]|[25, 52, 164, 240...|       0|
|[471, 496]|[39, 120, 124, 20...|       0|
|[471, 496]|[35, 249, 674, 71...|       0|
|[471, 496]|[39, 422, 449, 70...|       0|
|[471, 496]|[15, 229, 262, 28...|       0|
|[471, 496]|[26, 104, 143, 32...|       0|
|[471, 496]|[7, 185, 214, 350...|       0|
|[471, 496]|          [227, 390]|       0|
|[471, 496]|[71, 192, 208, 27...|       1|
|[471, 496]|[183, 193, 217, 2...|       1|
|[471, 496]|[161, 175, 177, 4...|       0|
|[471, 496]|[125, 130, 327, 6...|       0|
|[471, 496]|[392, 461, 569, 8...|       0|
|[471, 496]|[27, 78, 104, 177...|       0|
|[471, 496]|[101, 147, 229, 3...|       0|
|[471, 496]|[71, 208, 217, 26...|       0|
|[471, 496]|[43, 70, 176, 204...|       0|
|[471, 496]|  [25, 52, 278, 730]|       0|
|[471, 496]|[151, 432, 504, 8...|       0|
|[471, 496]|[71, 73, 118, 274...|       0|
+----------

dupletonDF = [candidate2: array<int>, items: array<int> ... 1 more field]


lastException: Throwable = null
contains_all: (x: org.apache.spark.sql.Column, y: org.apache.spark.sql.Column)org.apache.spark.sql.Column


[candidate2: array<int>, items: array<int> ... 1 more field]

In [17]:
val sqlContext = new SQLContext(sc)
val frequent_dupl = sqlContext.sql("SELECT d.items, d.candidate2, d.dupleton FROM dupletons d WHERE d.dupleton == 2")
frequent_dupl.show()

+--------------------+----------+--------+
|               items|candidate2|dupleton|
+--------------------+----------+--------+
|[8, 78, 427, 450,...|[471, 496]|       2|
|[6, 368, 471, 475...|[471, 496]|       2|
|[185, 471, 496, 6...|[471, 496]|       2|
|[90, 185, 239, 24...|[471, 496]|       2|
|[97, 100, 112, 12...|[471, 496]|       2|
|[177, 197, 242, 3...|[471, 496]|       2|
|[71, 145, 172, 27...|[471, 496]|       2|
|[68, 145, 217, 24...|[471, 496]|       2|
|[64, 95, 242, 307...|[471, 496]|       2|
|[68, 217, 334, 34...|[471, 496]|       2|
|[39, 177, 471, 49...|[471, 496]|       2|
|[242, 392, 461, 4...|[471, 496]|       2|
|[142, 145, 171, 2...|[471, 496]|       2|
|[132, 161, 242, 2...|[471, 496]|       2|
|[101, 114, 145, 1...|[471, 496]|       2|
|[32, 112, 181, 24...|[471, 496]|       2|
|[368, 471, 496, 5...|[471, 496]|       2|
|[27, 89, 177, 198...|[471, 496]|       2|
|[110, 355, 438, 4...|[471, 496]|       2|
|[18, 129, 185, 23...|[471, 496]|       2|
+----------

sqlContext = org.apache.spark.sql.SQLContext@70eb1a56
frequent_dupl = [items: array<int>, candidate2: array<int> ... 1 more field]


[items: array<int>, candidate2: array<int> ... 1 more field]

In [20]:
//val freq_dupletons = frequent_dupl.groupBy("candidate2").agg(count("*")as "support").where(col("support") > support)
//freq_dupletons.show

lastException: Throwable = null


Name: Syntax Error.
Message: 
StackTrace: 

In [56]:
case class duples(items: Array[Int], candidate2:Array[Int], support: Int)
val dupleRDD = frequent_dupl.rdd.map(r => (r(0), 1))
    .reduceByKey(_ + _)
dupleRDD.collect.foreach(println)

lastException = null


Name: org.apache.spark.SparkException
Message: Job 37 cancelled as part of cancellation of all jobs
StackTrace:   at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1889)
  at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1824)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$doCancelAllJobs$1.apply$mcVI$sp(DAGScheduler.scala:830)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$doCancelAllJobs$1.apply(DAGScheduler.scala:830)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$doCancelAllJobs$1.apply(DAGScheduler.scala:830)
  at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
  at org.apache.spark.scheduler.DAGScheduler.doCancelAllJobs(DAGScheduler.scala:830)
  at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2082)
  at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2059)


In [ ]:
//archive:


In [ ]:
//k = 1
val k = 1
val singleton_c = dataRDD.map(r=> r.map(item => (item, 1)))
    .flatMap(y=>y).reduceByKey(_ + _)
    .map(d => candidate(d._1, d._2))
val singleton = singleton_c.filter(s => s.support >= support)


In [ ]:
val k_ = 2
//val dupletons = singleton.map(s => s.value).collect.toSet.subsets(k_).copyToArray(d2)
case class candidate2(value: Array[Int], support: Int)
val candidat = singleton.map(s => candidate2(Array(s.value), s.support))




val singletonDF = spark.createDataFrame(singleton) 
val itemsDF = spark.createDataFrame(itemsRDD)
//itemsDF.show(200)

singletonDF.createOrReplaceTempView("single")
itemsDF.createOrReplaceTempView("data")
val sqlContext = new SQLContext(sc)
val dup = sqlContext.sql("SELECT s1.value as v1, s2.value as v2, d.items, d.row FROM data d, single s1, single s2 WHERE s1.value != s2.value AND array_contains(d.items, s1.value) AND array_contains(d.items, s2.value)")
dup.show()
println(dup.count())


val result = dup.rdd.map(d => ((d(0), d(1)), 1)).reduceByKey(_ + _).filter(d => d._2 > support)
result.collect.foreach(println)

In [ ]:
import org.apache.spark.sql.types._



val singleDF = spark.createDataFrame(candidat) 
singleDF.show()

val crossDF = itemsDF.crossJoin(singleDF)
crossDF.show()

val dupleDF = crossDF.withColumn(
    "singleton",
    array_intersect( col("value"), col("items"))
)
dupleDF.show()
dupleDF.createOrReplaceTempView("dupz")
val dup_filtered = sqlContext.sql("SELECT d.items, d.row, d.singleton FROM dupz d WHERE size(d.singleton) > 0 ")
dup_filtered.show()

import org.apache.spark.sql.functions.count

dup_filtered.groupBy("singleton").agg(count("row")as "support").where(col("support") > support).show

//println(dup_filtered.count())


In [ ]:
def generateCandidates(row: Set[Int], candidates: Set[Int], k: Int): Set[(Set[Int],Int)] = {
    val possible = candidates.subsets(k)
    val c = possible.map(p => if(p.subsetOf(row)) (p,1) else (p,0) ).toSet
    c
}



val data = dataRDD.map(d => generateCandidates(d, candidates, 2))
data.collect.foreach(println)
//apriori_k(1, support, candidates, dataRDD)

In [ ]:
def apriori_k(k: Int, s: Float, candidates: RDD[Int], data: Any): Any = {
    if( candidates.size < 2) return data
    
    C_t = generateCandidates(k, candidates, s)
    println(k)
    apriori_k(k+1, s, sc.parallelize(candidates.take((candidates.count()-1).toInt)), data ++ C_t)
}